In [12]:
!pip install strands-agents[mistral] python-dotenv

In [1]:
import json
import os
import sys
import dotenv
import boto3
import requests

import pprint

import yaml
from pathlib import Path
from typing import Dict, List, Optional, Tuple, TypeVar
from tqdm import tqdm

# Pydantic for structured data
from pydantic import BaseModel, Field

# Strands for AI agents
from strands.agent import Agent
from strands.models.mistral import MistralModel

# AWS authentication
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest

sys.path.append('..')

from src.utils import (
    save_json,
    read_json,
    load_file_content,
    get_job_paths,
    get_training_paths,
    sanity_check,
	chat_with_persona,
    track_api_call,  # Cost tracking from utils
    print_cost_summary,  # Cost summary from utils
    reset_cost_tracker  # Reset cost tracker from utils
)

from src.my_utils import (
    display_markdown_file,
    call_mistral,
    get_agent,
    batch_extract,
    send_message_to_chat
)

from src.models.persona_info import PersonaInfo

from src.models.interview_info import(
    InterviewInfo,
    InverviewQualityInfo,
    InterviewAgentMessage
)
from src.prompts.interview_prompt import(
    JOB_EXTENSION_INTERVIEW_PROMPT,
    JOB_EXTENSION_INTERVIEW_QUALITY_CHECK_PROMPT
)

# Load API key from .env file
dotenv.load_dotenv("../env")

# Check if we're good to go
if not os.getenv("MISTRAL_API_KEY"):
    print("❌ No MISTRAL_API_KEY found!")
    print("Create an env file with your API key")
else:
    print("✅ API key found, we're ready to roll")

✅ API key found, we're ready to roll


In [2]:
MAX_TURNS_IN_INTERVIEW = 10

In [3]:
DATA_PERSONAS_INFO_DIR = Path('../data_personas_info')
DATA_JOBS_DIR = Path('../data_jobs')
DATA_TRAININGS_DIR = Path('../data_trainings')
DATA_INTERVIEWS_DIR = Path('../data_interviews')

In [4]:
with open("../src/config.yaml", "r") as f:
    config = yaml.safe_load(f)

personas_info_data_version = config["personas_info_data_version"]
print(f"personas_info_data_version version: {personas_info_data_version}")

job_data_version = config["job_data_version"]
print(f"job_data_version version: {job_data_version}")

training_data_version = config["training_data_version"]
print(f"training_data_version version: {training_data_version}")

interview_data_version = config["interview_data_version"]
print(f"interview_data_version version: {interview_data_version}")

personas_info_data_version version: v14
job_data_version version: v4
training_data_version version: v7
interview_data_version version: v8


In [5]:
filename = f"map_clusters_jobs_{job_data_version}.json"
save_path = DATA_JOBS_DIR / filename
jobs_map = read_json(save_path)

In [6]:
# Load skills domains data
filename = f"final_map_clusters_trainings_{training_data_version}.json"
save_path = DATA_TRAININGS_DIR / filename
trainings_map = read_json(save_path)

print(f"✅ Loaded {len(trainings_map)} skills domains")
print("\n" + "="*50)

✅ Loaded 12 skills domains



In [7]:
# Load interviews
filename = f"interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
initial_interviews = read_json(interviews_save_path)

In [8]:
# Load Personas data
filename = f"aging_filter_personas_info_{personas_info_data_version}.json"
personas_save_path = DATA_PERSONAS_INFO_DIR / filename

personas_data = read_json(personas_save_path)

# Convert to PersonaInfo objects
personas = {
    pid: PersonaInfo.model_validate_json(data)
    for pid, data in personas_data.items()
}

print(f"✅ Loaded {len(personas)} personas")
print("\n" + "="*50)

✅ Loaded 100 personas



In [9]:
# Load jobs domains map data
filename = f"map_clusters_jobs_{job_data_version}.json"
save_path = DATA_JOBS_DIR / filename
jobs_map = read_json(save_path)

# print(jobs_map)

# Perform interviews

In [10]:
def conduct_persona_interview(
    persona_id: str,
    domains_str: str,
    conversation_id: str = None,
    max_turns: int = 5,
    model: str = "mistral-medium-latest",
    print_conversation: bool = False
) -> List[str]:
    """Interview a persona and return conversation transcript"""

    interview = InterviewInfo()

    # prompt = INITIAL_INTERVIEW_PROMPT
    prompt = JOB_EXTENSION_INTERVIEW_PROMPT

    interview_agent = get_agent(prompt, model_id=model)

    # Start with greeting
    agent_message = "In a single sentence, which domains you are interested in to find a job from follwing list:\n"
    agent_message += domains_str
    agent_message += "If none of them or not interested by a job, just say it"

    # print(conversation_id)
    # print(prompt)
    # print(agent_message)
    # return None

    if print_conversation:
        print(f"Assistant: {agent_message}")
    interview.interview.append(f"Assistant: {agent_message}")

    # Conduct interview
    for turn in range(max_turns):
        resp = send_message_to_chat(agent_message, persona_id, conversation_id)

        if resp is None:
            break

        user_response, conversation_id = resp
        interview.conversation_id = conversation_id
        interview.interview.append(f"User: {user_response}")
        if print_conversation:
            print(f"User: {user_response}")

        # Generate next question
        conversation_str = '\n'.join(interview.interview)
        agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, prompt=conversation_str)
        # agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, user_response)

        # Track cost (using utils.py function)
        # track_api_call(agent_response, model)

        if agent_response.conversation_finished is True:
            break
            
        agent_message = agent_response.message
        interview.interview.append(f"Assistant: {agent_message}")
        if print_conversation:
            print(f"Assistant: {agent_message}")

    return interview

In [13]:
# Interview all personas
#persona_ids = [f'persona_{i:03}' for i in range(1, 4)]
persona_ids = [f'persona_{i:03}' for i in range(1, 101)]

# personas_save_path = SUBMISSION_DIR / 'personas.json'
filename = f"job_extension_interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename

if not interviews_save_path.exists():
    save_json(interviews_save_path, {})

interviews = read_json(interviews_save_path)
personas_to_process = len(persona_ids) - len(interviews)
print(f'Personas to process: {personas_to_process}')

# Track how many new personas we process
new_personas_processed = 0

for persona_id in tqdm(persona_ids):

    persona_info = personas[persona_id]
    if persona_info.recommendation_type != 'jobs_trainings':
        continue

    print(persona_id)
    if persona_id in interviews:
        interview = interviews[persona_id]['interview']
        if len(interview) > 1:
            continue

    initial_interview = initial_interviews[persona_id]
    conversation_id = initial_interview['conversation_id']

    new_personas_processed += 1

    domains_str = ""
    for domain in jobs_map:
        domains_str += f"- {domain} : {jobs_map[domain]['description']}" + "\n"
    
    # Interview
    conversation = conduct_persona_interview(
        persona_id,
        domains_str,
        conversation_id=conversation_id,
        max_turns=10,
        print_conversation=True)
    interviews[persona_id] = conversation.model_dump()

    # Save every interview
    if len(interviews) % 1 == 0:
        save_json(interviews_save_path, interviews)

    # Show cost update every 20 personas
    # if new_personas_processed > 0 and new_personas_processed % 20 == 0:
    #     print(f"\n💰 Cost update after {new_personas_processed} new personas:")
    #     print_cost_summary()
    #     print()

    if new_personas_processed > 0:break

save_json(interviews_save_path, interviews)

print(f"\n✅ Interviewed {len(interviews)} personas total ({new_personas_processed} new)")

# Final cost summary for persona processing
if new_personas_processed > 0:
    print("\n📊 Persona processing costs:")
    print_cost_summary()

Personas to process: 37


100%|██████████| 100/100 [00:00<00:00, 85931.24it/s]

persona_001
persona_003
persona_004
persona_007
persona_010
persona_011
persona_013
persona_016
persona_017
persona_018
persona_019
persona_020
persona_021
persona_022
persona_023
persona_026
persona_027
persona_029
persona_030
persona_031
persona_033
persona_037
persona_038
persona_041
persona_043
persona_044
persona_045
persona_048
persona_049
persona_051
persona_056
persona_057
persona_058
persona_060
persona_061
persona_063
persona_066
persona_067
persona_069
persona_070
persona_071
persona_072
persona_074
persona_075
persona_076
persona_077
persona_078
persona_079
persona_080
persona_081
persona_082
persona_083
persona_084
persona_086
persona_087
persona_088
persona_089
persona_091
persona_093
persona_094
persona_096
persona_097
persona_098

✅ Interviewed 63 personas total (0 new)


---

# For Debug Only

---

# Redo interview of a Persona

In [13]:
if True:
    persona_id = 'persona_001'
    #filename = f"job_extension_interviews_{interview_data_version}.json"
    #interviews_save_path = DATA_INTERVIEWS_DIR / filename
    #interviews = read_json(interviews_save_path)

    filename = f"full_interviews_{persona_id}.json"
    interview_save_path = DATA_INTERVIEWS_DIR / filename
    interview = read_json(interview_save_path)
    #interviews[persona_id] = conversation.model_dump()
    #save_json(interviews_save_path, interviews)

    persona_info = personas[persona_id]

    if persona_info.recommendation_type != 'jobs_trainings':
        print("ISSUE")
    else:
        #initial_interview = initial_interviews[persona_id]
        #conversation_id = initial_interview['conversation_id']
        #conversation_id = None
        #initial_interview = interviews[persona_id]
        initial_interview = interview
        conversation_id = initial_interview['conversation_id']

        domains_str = ""
        for domain in jobs_map:
            domains_str += f"- {domain} : {jobs_map[domain]['description']}" + "\n"

        # Interview
        conversation = conduct_persona_interview(
            persona_id,
            domains_str,
            conversation_id=conversation_id,
            max_turns=6,
            print_conversation=True)

        #print(conversation)

        # interviews[persona_id] = conversation.model_dump()
        new_interview = conversation.model_dump()
        interview['interview'].extend(new_interview['interview'])
        
        #save_json(interviews_save_path, interviews)
        save_json(interview_save_path, interview)


Assistant: In a single sentence, which domains you are interested in to find a job from follwing list:
- Financial Operations And Compliance Management : Managing financial activities, ensuring regulatory compliance, and supporting risk management in finance.
- Legal Research And Procedural Support : Conducting legal research, drafting documents, and supporting procedural compliance and case management.
- Insurance Operations And Compliance Management : Managing insurance processes, ensuring compliance, assessing risks, and handling client services.
- Maritime And Port Operations Management : Managing and coordinating vessel, terminal, and cargo operations in maritime and port environments.
- Industrial Production And Live Event Technical Operations : Encompasses equipment design, maintenance, and coordination in industrial and live event settings.
- Health Safety And Environmental Compliance : Ensuring workplace safety, regulatory compliance, and environmental health in industrial set

/tmp/ipykernel_21348/1493981661.py:47: DeprecationWarning: Agent.structured_output method is deprecated. You should pass in `structured_output_model` directly into the agent invocation. see: https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/structured-output/
  agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, prompt=conversation_str)


Assistant: Are you open to relocating for a job? What is your highest level of completed education according to the Brazilian education system: Ensino Fundamental, Ensino Médio, Técnico, Tecnólogo, Graduação, Bacharelado, Licenciatura, Pós-graduação, Especialização, Mestrado, MBA, Doutorado?
User: no relocating. i stay são paulo.
my school is ensino fundamental. but i lern fast!
Assistant: How many years of experience do you have in total and specifically in Food Production Quality and Safety Management? Do you know any other languages? If so, which ones and what is your proficiency level: Nenhum, Básico, Intermediário, Avançado?
User: i got no years. zero. but i watch videos of food factorys.
only portuges. basico english like this.
Assistant: What are your career goals in Food Production Quality and Safety Management?
User: i wanna start with trainig. lern how to keep food safe and clean. maybe one day work in factory but first need to lern the rules. they got rules for everything ri

# Check interview qualities

In [32]:
filename = f"job_extension_interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)

In [33]:
def check_interview_quality(
    interview: str,
    model: str = "mistral-small-latest",
    print_prompt: bool = False
) -> InverviewQualityInfo:
    prompt = JOB_EXTENSION_INTERVIEW_QUALITY_CHECK_PROMPT.format(
        interview=interview
    )

    if print_prompt is True:
        print(prompt)

    extraction_agent = get_agent(model_id=model, temperature=0.0)
    result = extraction_agent.structured_output(output_model=InverviewQualityInfo, prompt=prompt)

    return result

In [34]:
cache_period = 5

filename = f"quality_job_extension_interviews_{interview_data_version}.json"
save_path = DATA_INTERVIEWS_DIR / filename
if not save_path.exists():
    save_json(save_path, {})
quality_interviews = read_json(save_path)


new_items_processed = 0
#for persona_id in tqdm(interviews):
for persona_id in tqdm(personas_data):
    persona_data_dict = json.loads(personas_data[persona_id])
    if persona_data_dict['recommendation_type'] != 'jobs_trainings':
        continue

    if persona_id not in interviews:
        quality = InverviewQualityInfo(
            {
                'quality_level': 'NOK',
                'rationale': 'interview missing'
            }
        )
        quality_str = json.dumps(quality.model_dump(), ensure_ascii=False)
        quality_interviews[persona_id] = quality_str
        save_json(save_path, quality_interviews)
        continue

    if persona_id in quality_interviews:
        quality = json.loads(quality_interviews[persona_id])
        if quality['quality_level'] == 'OK':
            continue

    new_items_processed = new_items_processed + 1

    interview = interviews[persona_id]['interview']

    interview_str = "\n".join(interview)
    # print(interview_str)

    quality = check_interview_quality(
        interview_str,
        model='mistral-medium-latest',
        print_prompt=False)
    quality_str = json.dumps(quality.model_dump(), ensure_ascii=False)

    quality_interviews[persona_id] = quality_str

    if new_items_processed % cache_period == 0:
        save_json(save_path, quality_interviews)

    # if new_items_processed > 0:break

save_json(save_path, quality_interviews)

  0%|          | 0/100 [00:00<?, ?it/s]


Analyse the following interview.
Your goal is to classify an interview so that to decide if it can help to perform all persona information extraction form this list:
- Open to relocate for work
- Education level: Their highest completed education level (one of: Ensino Fundamental, Ensino Médio, Técnico, Tecnólogo, Graduação, Bacharelado, Licenciatura, Pós-graduação, Especialização, Mestrado, MBA, Doutorado)
- Years of experience
- Languages: Languages they speak
- Goals: Their stated career or learning objectives. Make sure to capture all meaningful informations, but limit up to 20 words.
- Target domains: Professional domains/sectors they're interested in.

You must reply with the following format:
- quality_level: OK/NOK
    - if information are missing reply NOK
    - Also, if you see that Assistant have hallucinated informations about the persona, reply NOK
- rationale: short justification of the quelity level decision

INTERVIEW:
Assistant: In a single sentence, which domains you

  7%|▋         | 7/100 [00:02<00:32,  2.85it/s]


Analyse the following interview.
Your goal is to classify an interview so that to decide if it can help to perform all persona information extraction form this list:
- Open to relocate for work
- Education level: Their highest completed education level (one of: Ensino Fundamental, Ensino Médio, Técnico, Tecnólogo, Graduação, Bacharelado, Licenciatura, Pós-graduação, Especialização, Mestrado, MBA, Doutorado)
- Years of experience
- Languages: Languages they speak
- Goals: Their stated career or learning objectives. Make sure to capture all meaningful informations, but limit up to 20 words.
- Target domains: Professional domains/sectors they're interested in.

You must reply with the following format:
- quality_level: OK/NOK
    - if information are missing reply NOK
    - Also, if you see that Assistant have hallucinated informations about the persona, reply NOK
- rationale: short justification of the quelity level decision

INTERVIEW:
Assistant: In a single sentence, which domains you

 20%|██        | 20/100 [00:03<00:12,  6.25it/s]

...

 44%|████▍     | 44/100 [00:05<00:06,  8.92it/s]


Analyse the following interview.
Your goal is to classify an interview so that to decide if it can help to perform all persona information extraction form this list:
- Open to relocate for work
- Education level: Their highest completed education level (one of: Ensino Fundamental, Ensino Médio, Técnico, Tecnólogo, Graduação, Bacharelado, Licenciatura, Pós-graduação, Especialização, Mestrado, MBA, Doutorado)
- Years of experience
- Languages: Languages they speak
- Goals: Their stated career or learning objectives. Make sure to capture all meaningful informations, but limit up to 20 words.
- Target domains: Professional domains/sectors they're interested in.

You must reply with the following format:
- quality_level: OK/NOK
    - if information are missing reply NOK
    - Also, if you see that Assistant have hallucinated informations about the persona, reply NOK
- rationale: short justification of the quelity level decision

INTERVIEW:
Assistant: In a single sentence, which domains you

 45%|████▌     | 45/100 [00:07<00:10,  5.36it/s]


Analyse the following interview.
Your goal is to classify an interview so that to decide if it can help to perform all persona information extraction form this list:
- Open to relocate for work
- Education level: Their highest completed education level (one of: Ensino Fundamental, Ensino Médio, Técnico, Tecnólogo, Graduação, Bacharelado, Licenciatura, Pós-graduação, Especialização, Mestrado, MBA, Doutorado)
- Years of experience
- Languages: Languages they speak
- Goals: Their stated career or learning objectives. Make sure to capture all meaningful informations, but limit up to 20 words.
- Target domains: Professional domains/sectors they're interested in.

You must reply with the following format:
- quality_level: OK/NOK
    - if information are missing reply NOK
    - Also, if you see that Assistant have hallucinated informations about the persona, reply NOK
- rationale: short justification of the quelity level decision

INTERVIEW:
Assistant: In a single sentence, which domains you

100%|██████████| 100/100 [00:08<00:00, 11.49it/s]


In [36]:
filename = f"quality_job_extension_interviews_{interview_data_version}.json"
save_path = DATA_INTERVIEWS_DIR / filename
quality_interviews = read_json(save_path)

for persona_id in quality_interviews:
    quality = json.loads(quality_interviews[persona_id])
    if quality['quality_level'] != 'OK':
        print(persona_id)
        print(quality['rationale'])
        print('---')

persona_007
The interview lacks specific details necessary to extract all required persona information. The user's responses are poetic and metaphorical, making it difficult to determine clear answers for education level, years of experience, and language proficiency. Additionally, there is no clear information about the user's goals or target domains for employment.
---
persona_030
The user is not interested in a job, and the interview does not provide any information about the persona's education level, years of experience, languages spoken, goals, or willingness to relocate for work.
---
persona_044
The interview lacks information on several key points: openness to relocate for work, education level, years of experience, and languages spoken. Additionally, while goals and target domains are partially covered, they are not fully detailed.
---
persona_045
The interview does not provide sufficient information to extract all required persona details, particularly regarding relocation, e

# Debug

In [ ]:
if False:
    persona_id = 'persona_001'

    persona_info = personas[persona_id]
    if persona_info.recommendation_type == 'jobs_trainings':
        print('HERE')
    
    if False:
        persona_id = 'persona_001'
        initial_interview = initial_interviews[persona_id]
        conversation_id = initial_interview['conversation_id']
        conversation = initial_interview['interview']
        # print(conversation_id)
        # print(conversation)
    
        domains_str = ""
        for domain in jobs_map:
            domains_str += f"- {domain} : {jobs_map[domain]['description']}" + "\n"
            
        interview = conduct_persona_interview(
            persona_id,
            domains_str,
            conversation_id = conversation_id,
            max_turns=5,
            model="mistral-small-latest",
            print_conversation=True
        )
        
        print(interview)

In [ ]:
if False:
    filename = f"awareness_extension_interviews_{interview_data_version}.json"
    interviews_save_path = DATA_INTERVIEWS_DIR / filename
    if not interviews_save_path.exists():
        save_json(interviews_save_path, {})
    interviews = read_json(interviews_save_path)
    
    interviews[persona_id] = interview.model_dump()
    save_json(interviews_save_path, interviews)
